# Choosing between torch.nn.functional vs torch.nn module

Credit goes to forum users:

Miguel Varela Ramos

for their responses.

In [16]:
import torch
import torch.nn as nn

import torch.nn.functional as F

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In the `__init__` function, you are supposed to initialize the layers you want to use. Unlike keras, Pytorch goes more low level and you have to specify the sizes of your network so that everything matches.

In the `forward` method, you specify the connections of your layers. This means that you will use the layers you already initialized, in order to re-use the same layer for each forward pass of data you make.

`torch.nn.Functional` contains some useful functions like activation functions a convolution operations you can use. However, `torch.nn.Function` are **not full layers** so if you want to specify a layer of any kind you should use `torch.nn.Module`.

You would use the `torch.nn.Functional` conv operations to define a custom layer for example with a convolution operation, but not to define a standard convolution layer.

A `nn.Module` is actually a OO wrapper around the functional interface, that contains a number of utility methods, like eval() and parameters(), and it automatically creates the parameters of the modules for you.
you can use the functional interface whenever you want, but that requires you to define the weights by hand. 

In [9]:
# Dummy example where dropout uses the training parameter
sample = torch.tensor([1, 2, 3])
torch.nn.functional.dropout(sample, p=0.5, training=False, inplace=False);

From Adam Paszke, lead Pytorch Dev:
> I think it’s clearer to use modules for stateful function (in this case dropout can be considered stateful, because of this flag), and functional for everything else.

## Examples of nn.module

`torch.nn.Sequential` A sequential container. Modules will be added to it in the order they are passed in the constructor.

In [11]:
layer_1 = nn.Sequential(nn.Conv2d(1, 10, 3),
                            nn.ReLU(),
                            nn.Conv2d(10, 64, 3),
                            nn.ReLU()
                           )

In [12]:
layer_1

Sequential(
  (0): Conv2d(1, 10, kernel_size=(3, 3), stride=(1, 1))
  (1): ReLU()
  (2): Conv2d(10, 64, kernel_size=(3, 3), stride=(1, 1))
  (3): ReLU()
)

## Two ways of writing a sample model using Sequential

In [15]:
class ConvNetSequential(nn.Module):
    
    def __init__(self, num_classes, kernel_size=5, stride=1, padding=2):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size, stride, padding),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size, stride, padding),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        output = self.layer1(x)
        output = self.layer2(output)
        output = output.reshape(output.size(0), -1)
        output = self.fc(output)
        return output

In [ ]:
class ConvNet(nn.Module):
    
    def __init__(self, num_classes, kernel_size=5, stride=1, padding=2):
        super().__init__()
        self.layer1 = nn.Conv2d(1, 16, kernel_size, stride, padding)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.layer2 = nn.Conv2d(16, 32, kernel_size, stride, padding)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        output = self.pool1(F.relu(F.batch_norm(self.layer1(x))))
        output = self.pool2(F.relu(F.batch_norm(output)))
        output = output.reshape(output.size(0), -1)
        output = self.fc(output)
        return output